In [1]:
import pandas as pd


In [2]:
movies_df = pd.read_csv('data/movies_data/movies.csv', sep=',')

In [5]:
ratings1_df = pd.read_csv('data/movies_data/ratings1.csv', sep=',')
ratings2_df = pd.read_csv('data/movies_data/ratings2.csv', sep=',')

In [6]:
dates_df = pd.read_csv('data/movies_data/dates.csv', sep=',')

In [3]:
print(movies_df['title'].nunique())

9737


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [19]:
dates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    100836 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 787.9 KB


In [13]:
print(ratings1_df['userId'].nunique())

274


In [18]:
dates_df['date'] = pd.to_datetime(dates_df['date'])

In [21]:
dates_df['date'].dt.year.value_counts()

2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: date, dtype: int64

In [23]:
ratings_df = pd.concat([ratings1_df, ratings2_df], ignore_index=True)
display(ratings_df)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [24]:
print('Число строк в таблице ratings: ', ratings_df.shape[0])
print('Число строк в таблице dates: ', dates_df.shape[0])
print(ratings_df.shape[0] == dates_df.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


In [25]:
print(ratings1_df.tail(1))
print(ratings2_df.head(1))

       userId  movieId  rating
40000     274     5621     2.0
   userId  movieId  rating
0     274     5621     2.0


In [26]:
ratings_df = ratings_df.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings_df.shape[0])

Число строк в таблице ratings:  100836


In [27]:
ratings_dates = pd.concat([ratings_df, dates_df], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


In [30]:
joined = ratings_dates.join(
    movies_df.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [32]:
merged = ratings_dates.merge(
    movies_df,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [33]:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])


Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


In [34]:
import re 

In [36]:
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [39]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   userId        100836 non-null  int64         
 1   movieId       100836 non-null  int64         
 2   rating        100836 non-null  float64       
 3   date          100836 non-null  datetime64[ns]
 4   title         100836 non-null  object        
 5   genres        100836 non-null  object        
 6   year_release  100818 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 6.2+ MB


In [38]:
merged['year_release'] = merged['title'].apply(get_year_release)
merged.head()

,userId,movieId,rating,date,title,genres,year_release
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0


In [42]:
mask1 = merged['year_release'] == 1999
print(merged[mask1]['rating'].min())

0.5
